<a href="https://colab.research.google.com/github/RafaelBNN/if697/blob/main/Projeto_2_IF697.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando as Bibliotecas

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# Blbliotecas e funcoes relacionadas ao mlflow
!pip install mlflow
import mlflow
from pprint import pprint
!pip install utils

def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

# 1. Introdução

Este Notebook busca dar continuidade à análise do dataset estudado no primeiro projeto. Lá, exploramos e tratamos um dataset relacionado a competições de powerlifting, sobre o qual foi realizado um pré-processeamento e uma análise descritiva de dados. 

Neste projeto, daremos prosseguimento a isso aplicando a ideia de *predição* dos dados. 

Os problemas de predição aparecem quando queremos descobrir um atributo desconhecido de um registro a partir de outros atributos desse registro. Esses problemas são, principalmente, divididos em duas categorias: problemas de *classificação* e problemas de *regressão*. Na *classificação*, o objetivo é predizer um atributo categórico de um registro, como o sexo de um indivíduo ou a espécie de um animal, por exemplo, enquanto que, na *regressão*, queremos prever um atributo contínuo, como a altura ou o peso de uma pessoa.

A partir do dataset observado, portanto, vamos aplicar o conceito de regressão ao atributo `BodyweightKg` (o peso corporal do competidor). Vamos treinar alguns modelos a partir de variados algoritmos de *machine learning* para realizar a predição dessa coluna e avaliar a performance de cada um desses modelos.

O dataframe que será utilizado na predição é uma amostra do dataset original. Para a previsão, serão utilizadas as colunas `Sex`, o sexo do competidor; `Age`, a idade do competidor; `Best3SquatKg`, a melhor tentativa dentre as 3 no agachamento; `Best3SBenchKg`, a melhor tentativa dentre as 3 no supino; e `Best3DeadliftKg`, a melhor tentativa dentre as 3 no levantamento terra.

#### Importação do Dataframe

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/RafaelBNN/datasets/main/powerlifting_final.csv')

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName,Best3SquatKg_norm,Best3BenchKg_norm,Best3DeadliftKg_norm,TotalKg_norm,BodyweightKg_norm,IPFPoints_norm,TotalKg_disc,lof_outlier,Age_disc
0,0,692083,Hayden Fulwood,1,SBD,Raw,18.000000,18-19,MR-T3,88.0,90,NaN,NaN,NaN,167.50,NaN,NaN,NaN,122.50,NaN,NaN,NaN,227.50,517.50,1,334.25,354.31,322.27,482.79,Yes,USA,USAPL,2013-04-27,USA,AL,S.E HS State Wars,-0.385417,-0.528924,0.080000,-0.156780,-0.411065,-0.206464,"(507.5, 576.06]",1,"(15.9, 19.6]"
1,1,621012,Jozef Slimák,1,B,Multi-ply,46.000000,45-49,M-M2,82.2,82.5,NaN,NaN,NaN,NaN,165.0,180.0,-190.0,180.00,NaN,NaN,NaN,NaN,180.00,1,120.85,129.06,116.68,571.78,NaN,Slovakia,WPC,2012-11-06,USA,NV,World Championships,NaN,-0.280420,NaN,-0.728814,-0.464106,-0.037484,"(142.5, 202.5]",1,"(45.5, 49.2]"
2,2,164449,Chase Garrett,1,D,Raw,14.000000,13-15,Teenage 13-15,44.0,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.31,70.31,1,83.29,98.28,82.94,212.28,NaN,NaN,SPF,2008-12-06,USA,NaN,Body by George Classic,NaN,NaN,-0.758347,-0.914729,-0.813443,-0.720126,"(19.999, 142.5]",1,"(12.2, 15.9]"
3,3,353419,Edgar Rodea,1,SBD,Single-ply,27.995064,NaN,Boys,92.9,99.7,NaN,NaN,NaN,204.12,NaN,NaN,NaN,129.27,NaN,NaN,NaN,185.97,519.36,1,326.42,326.42,314.34,420.04,Yes,NaN,THSPA,2017-01-28,USA,TX,Santo Meet,-0.232833,-0.499665,-0.141493,-0.153627,-0.366255,-0.325618,"(507.5, 576.06]",1,"(27.0, 30.7]"
4,4,128144,K. Kaufman,1,B,Raw,46.228441,NaN,Open,90.0,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.15,NaN,NaN,NaN,NaN,145.15,6,92.66,92.66,89.30,558.46,NaN,NaN,USPF,1979-12-08,USA,ND,3rd Annual Midwestern Bench Press Championships,NaN,-0.431035,NaN,-0.787881,-0.392775,-0.062777,"(142.5, 202.5]",1,"(45.5, 49.2]"


In [4]:
df.dtypes

Unnamed: 0                int64
Unnamed: 0.1              int64
Name                     object
Sex                       int64
Event                    object
Equipment                object
Age                     float64
AgeClass                 object
Division                 object
BodyweightKg            float64
WeightClassKg            object
Squat1Kg                float64
Squat2Kg                float64
Squat3Kg                float64
Best3SquatKg            float64
Bench1Kg                float64
Bench2Kg                float64
Bench3Kg                float64
Best3BenchKg            float64
Deadlift1Kg             float64
Deadlift2Kg             float64
Deadlift3Kg             float64
Best3DeadliftKg         float64
TotalKg                 float64
Place                    object
Wilks                   float64
McCulloch               float64
Glossbrenner            float64
IPFPoints               float64
Tested                   object
Country                  object
Federati

In [5]:
# As colunas que serao utilizadas
df = df[['Sex','Age','Best3SquatKg','Best3BenchKg','Best3DeadliftKg','BodyweightKg']]

df.head()

,Sex,Age,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,BodyweightKg
0,1,18.000000,167.50,122.50,227.50,88.0
1,1,46.000000,NaN,180.00,NaN,82.2
2,1,14.000000,NaN,NaN,70.31,44.0
3,1,27.995064,204.12,129.27,185.97,92.9
4,1,46.228441,NaN,145.15,NaN,90.0


In [6]:
len(df)

16676

In [7]:
df.isnull().sum(axis = 0)/len(df)

Sex                0.000000
Age                0.000000
Best3SquatKg       0.243404
Best3BenchKg       0.042756
Best3DeadliftKg    0.190094
BodyweightKg       0.005937
dtype: float64

In [8]:
# Para simplificar, vamos executar um dropna sobre o dataframe

df = df.dropna(subset=['Sex','Age','Best3SquatKg','Best3BenchKg','Best3DeadliftKg','BodyweightKg'])

len(df)

12476

In [9]:
# Alem disso, vamos normalizar as colunas que serao usadas, pois alguns algoritmos de previsao preferem dessa forma

df['Age'] = ((df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())*2) -1
df['Best3SquatKg'] = ((df['Best3SquatKg'] - df['Best3SquatKg'].min()) / (df['Best3SquatKg'].max() - df['Best3SquatKg'].min())*2) -1
df['Best3BenchKg'] = ((df['Best3BenchKg'] - df['Best3BenchKg'].min()) / (df['Best3BenchKg'].max() - df['Best3BenchKg'].min())*2) -1
df['Best3DeadliftKg'] = ((df['Best3DeadliftKg'] - df['Best3DeadliftKg'].min()) / (df['Best3DeadliftKg'].max() - df['Best3DeadliftKg'].min())*2) -1


# 2. Conjuntos de treinamento, validação e teste

Ao trabalhar com modelos de *machine learning*, é imprescindível que seja feita a divisão dos dados entre conjunto de treinamento, conjunto de validação e conjunto de teste (*training set*, *validation set* e *test set*). Essa divisão contribui para a avaliação de como o modelo está generalizando os dados a serem previstos e nos ajuda a identificar problemas como *overfitting*.

Aqui, vamos dividir o dataset utilizando a função `train_test_split` da biblioteca `sklearn`.

In [10]:
from sklearn.model_selection import train_test_split

# Os dataframes 'x' contem as colunas que serao utilizadas para realizar a predicao,
# e os dataframes 'y' contem a coluna sobre a qual sera realizada a predicao.

x_df = df.drop('BodyweightKg', axis=1)
y_df = df['BodyweightKg']

# Por padrao, o test set sera formado por 25% dos dados fornecidos, e o training set, 75%.
# Alem disso, o parametro random_state permite que a escolha dos conjuntos de treinamento e teste
# seja a mesma sempre que executarmos esta linha. 

x_train, x_test, y_train, y_test = train_test_split(x_df,y_df,random_state=10) 

In [11]:
x_train.head()

,Sex,Age,Best3SquatKg,Best3BenchKg,Best3DeadliftKg
10825,0,-0.586207,-0.635417,-0.738462,-0.352941
5873,1,-0.710345,-0.041667,0.076923,0.279412
1923,1,-0.451811,-0.125000,-0.123077,-0.058824
12586,1,-0.650272,-0.166708,-0.534031,-0.057176
13777,1,0.006897,-0.034417,-0.296800,0.236294


In [12]:
y_train.head()

10825     77.00
5873      89.95
1923      89.30
12586     53.80
13777    108.64
Name: BodyweightKg, dtype: float64

In [13]:
len(x_train), len(y_train), len(x_test), len(y_test)

(9357, 9357, 3119, 3119)

# 3. Modelos Iniciais

Após a divisão do dataset, agora empeçaremos a previsão da coluna `BodyweightKg`. Vamos aplicar alguns algoritmos de regressão aos conjuntos de treinamento e teste e avaliar a performance de cada um dos modelos treinados.

## *Autologging*

Antes de aplicar os algoritmos, porém, vamos chamar a função `mlflow.sklearn.autolog()`. Ela permite que os modelos sejam registrados automaticamente logo após sua execução utilizando a ferramenta *MLFlow*. Isso permite o rastreamento de como nosso programa está se comportando e como as predições evoluem ao longo do tempo.

Ao registrar os modelos automaticamente, são guardados os paraâmetros e o score daquele modelo para serem avaliados no futuro.

In [14]:
mlflow.sklearn.autolog()

## Regressão Linear

O primeiro algoritmo e o mais simples que vamos aplicar vai ser a simples regressão linear. Esse algoritmo de regressão busca encontrar a melhor reta que se ajusta aos dados, ou seja, a reta que minimiza a soma dos resíduos.

In [15]:
from sklearn.linear_model import LinearRegression

modelo1 = LinearRegression()

with mlflow.start_run() as run1:
    modelo1.fit(x_train,y_train)
    print("Logged data and model in run {}".format(run1.info.run_id))

y_pred1 = modelo1.predict(x_test)

2021/08/17 05:05:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Logged data and model in run adbc69061f084f74948e8a5da4ad3046


In [16]:
from sklearn.metrics import mean_squared_error

print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred1))

Mean squared error: 305.69


In [17]:
modelo1.score(x_test,y_test)

0.42925550418604164

## Decision Tree

Os algoritmos de árvore de decisão estão entre os algoritmos com melhor taxa de acurácia x explicabilidade. Ao passo que eles entregam uma acurácia considerável, esses algoritmos também têm uma boa taxa de interpretabilidade, o que significa que suas previsões são de fácil entendimento, diferente dos modelos que usam redes neurais, por exemplo.

Vamos, então, aplicar o algoritmo `DecisionTreeRegressor` e avaliar sua performance.

In [18]:
from sklearn import tree

modelo2 = tree.DecisionTreeRegressor(random_state=10)

with mlflow.start_run() as run2:
    modelo2.fit(x_train,y_train)
    print("Logged data and model in run {}".format(run2.info.run_id))

y_pred2 = modelo2.predict(x_test)

2021/08/17 05:05:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Logged data and model in run 9a3b86e49a184675a095aea55fe6a065


In [19]:
modelo2.score(x_test,y_test) # overfitting

-0.1190605397850142

Perceba que o modelo treinado acima não obteve um bom desempenho. Isso ocorreu porque o atributo `max_depth` da classe `DecisionTreeRegressor` é, por padrão, a maior profundidade possível, o que, nesse caso, levou a um overfitting.

Podemos, então, alterar o valor desse e de outros atributos para tentar obter uma performance mais próxima ao ideal.

In [20]:
# Aqui podemos alterar o atributo max_depth e o atributo min_split_samples (default=2), resultando num score diferente
modelo22 = tree.DecisionTreeRegressor(max_depth=5, random_state=10) 

with mlflow.start_run() as run22:
    modelo22.fit(x_train,y_train)
    print("Logged data and model in run {}".format(run22.info.run_id))

y_pred22 = modelo22.predict(x_test)

2021/08/17 05:05:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Logged data and model in run 95456caf547640ef863d264e3b8445a7


In [21]:
modelo22.score(x_test,y_test)

0.4250097031452772

Abaixo aplicamos `cross_val_score` aos dois modelos treinados.

In [22]:
from sklearn.model_selection import cross_val_score

cross_val_score(modelo2, x_train, y_train, cv=10) # modelo com overfit

2021/08/17 05:05:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'eb6c40a287074cbeb4fd9077c47a3198', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/08/17 05:05:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <htt

array([-0.1945758 , -0.23627933, -0.18019839, -0.38504958, -0.17041169,
       -0.15881799, -0.21863753, -0.30225223, -0.11893977, -0.0908888 ])

In [23]:
cross_val_score(modelo22, x_train, y_train, cv=10) # modelo com max_depth=5

2021/08/17 05:05:15 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '893a8354b3a34e17b90c139d8b2eb347', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/08/17 05:05:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <htt

array([0.41979619, 0.4097474 , 0.40820003, 0.36196559, 0.41220871,
       0.39218524, 0.40878043, 0.37763231, 0.41366703, 0.42547843])

## Random Forest

O método *Random Forest* é um tipo de algoritmo um tanto parecido com as árvores de decisão. Nele, são geradas múltiplas ávores de decisão que são treinadas e usadas para convergir a uma previsão final. Para treinar cada uma dessas árvores, o algoritmo pode usar partes diferentes dos dados do conjunto de treinamento ou até mesmo diferentes conjuntos de features dentre as que serão utilizadas na previsão.

In [24]:
from sklearn.ensemble import RandomForestRegressor

# O algoritmo de random forest tambem oferece os algortimos de max_depth e min_split_sample, que podem ser alterados para obter-se resultados diferentes
modelo3 = RandomForestRegressor(max_depth=5, random_state=10)

with mlflow.start_run() as run3:
    modelo3.fit(x_train,y_train)
    print("Logged data and model in run {}".format(run3.info.run_id))

y_pred3 = modelo3.predict(x_test)

2021/08/17 05:05:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Logged data and model in run f542b8f2f37a406c985f760d124edee4


In [25]:
modelo3.score(x_test,y_test)

0.44307796297090707

## Rede Neural

As redes neurais estão entre os mais citados e também mais precisos algoritmos de predição já criados. Seu poder vem do fato de esse tipo de algoritmo ser capaz de encontrar relações das mais altas ordens entre os atributos (diferente, por exemplo, da regressão linear, que é capaz apenas de considerar relações *lineares*). Outra característica que possibilita que as redes neurais sejam tão precisas é a quantidade de hiper-parâmetros, que podem ser alterados dependendo da tarefa de previsão a ser realizada.

Aqui, vamos aplicar o algoritmo `MLPRegressor` aos nossos conjuntos de treinamento e teste e avaliar sua performance com hiper-parâmetros *default*.

In [26]:
from sklearn.neural_network import MLPRegressor

with mlflow.start_run() as run4:
    modelo4 = MLPRegressor(random_state=10).fit(x_train, y_train)
    print("Logged data and model in run {}".format(run4.info.run_id))

y_pred4 = modelo4.predict(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
2021/08/17 05:05:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.ml

Logged data and model in run de2c854fc9254a008af501ca64142bc1


In [27]:
modelo4.score(x_test,y_test)

0.4352326053108301

# 4. MLFlow

Após treinar os modelos iniciais, vamos verificar os registros obtidos utilizando o MLFlow.

In [28]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run1.info.run_id)

print('Parametros utilizados:')
pprint(params)
print('\nMetricas:')
pprint(metrics)
print('\nAlgoritmo utilizado:')
pprint(tags)
print('\nArtifacts:')
pprint(artifacts)

Parametros utilizados:
{'copy_X': 'True',
 'fit_intercept': 'True',
 'n_jobs': 'None',
 'normalize': 'False'}

Metricas:
{'training_mae': 13.432513967102905,
 'training_mse': 304.3910590175906,
 'training_r2_score': 0.40325855635472385,
 'training_rmse': 17.446806556432914,
 'training_score': 0.40325855635472385}

Algoritmo utilizado:
{'estimator_class': 'sklearn.linear_model._base.LinearRegression',
 'estimator_name': 'LinearRegression'}

Artifacts:
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt']


In [29]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run2.info.run_id)

print('Parametros utilizados:')
pprint(params)
print('\nMetricas:')
pprint(metrics)
print('\nAlgoritmo utilizado:')
pprint(tags)
print('\nArtifacts:')
pprint(artifacts)

Parametros utilizados:
{'ccp_alpha': '0.0',
 'criterion': 'mse',
 'max_depth': 'None',
 'max_features': 'None',
 'max_leaf_nodes': 'None',
 'min_impurity_decrease': '0.0',
 'min_impurity_split': 'None',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'presort': 'deprecated',
 'random_state': '10',
 'splitter': 'best'}

Metricas:
{'training_mae': 0.0065362829966869734,
 'training_mse': 0.07224798546542698,
 'training_r2_score': 0.9998583619134995,
 'training_rmse': 0.2687898537248513,
 'training_score': 0.9998583619134994}

Algoritmo utilizado:
{'estimator_class': 'sklearn.tree._classes.DecisionTreeRegressor',
 'estimator_name': 'DecisionTreeRegressor'}

Artifacts:
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt']


In [30]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run22.info.run_id)

print('Parametros utilizados:')
pprint(params)
print('\nMetricas:')
pprint(metrics)
print('\nAlgoritmo utilizado:')
pprint(tags)
print('\nArtifacts:')
pprint(artifacts)

Parametros utilizados:
{'ccp_alpha': '0.0',
 'criterion': 'mse',
 'max_depth': '5',
 'max_features': 'None',
 'max_leaf_nodes': 'None',
 'min_impurity_decrease': '0.0',
 'min_impurity_split': 'None',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'presort': 'deprecated',
 'random_state': '10',
 'splitter': 'best'}

Metricas:
{'training_mae': 13.104485872151269,
 'training_mse': 291.1634615354983,
 'training_r2_score': 0.42919051257872787,
 'training_rmse': 17.06351257905295,
 'training_score': 0.42919051257872787}

Algoritmo utilizado:
{'estimator_class': 'sklearn.tree._classes.DecisionTreeRegressor',
 'estimator_name': 'DecisionTreeRegressor'}

Artifacts:
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt']


In [31]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run3.info.run_id)

print('Parametros utilizados:')
pprint(params)
print('\nMetricas:')
pprint(metrics)
print('\nAlgoritmo utilizado:')
pprint(tags)
print('\nArtifacts:')
pprint(artifacts)

Parametros utilizados:
{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'mse',
 'max_depth': '5',
 'max_features': 'auto',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_impurity_split': 'None',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '10',
 'verbose': '0',
 'warm_start': 'False'}

Metricas:
{'training_mae': 12.977786887496167,
 'training_mse': 284.61508137753196,
 'training_r2_score': 0.4420282412610841,
 'training_rmse': 16.870538858540705,
 'training_score': 0.44202824126108403}

Algoritmo utilizado:
{'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
 'estimator_name': 'RandomForestRegressor'}

Artifacts:
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt']


In [32]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run4.info.run_id)

print('Parametros utilizados:')
pprint(params)
print('\nMetricas:')
pprint(metrics)
print('\nAlgoritmo utilizado:')
pprint(tags)
print('\nArtifacts:')
pprint(artifacts)

Parametros utilizados:
{'activation': 'relu',
 'alpha': '0.0001',
 'batch_size': 'auto',
 'beta_1': '0.9',
 'beta_2': '0.999',
 'early_stopping': 'False',
 'epsilon': '1e-08',
 'hidden_layer_sizes': '(100,)',
 'learning_rate': 'constant',
 'learning_rate_init': '0.001',
 'max_fun': '15000',
 'max_iter': '200',
 'momentum': '0.9',
 'n_iter_no_change': '10',
 'nesterovs_momentum': 'True',
 'power_t': '0.5',
 'random_state': '10',
 'shuffle': 'True',
 'solver': 'adam',
 'tol': '0.0001',
 'validation_fraction': '0.1',
 'verbose': 'False',
 'warm_start': 'False'}

Metricas:
{'training_mae': 13.394924733268539,
 'training_mse': 302.5030589543456,
 'training_r2_score': 0.406959873623961,
 'training_rmse': 17.392615069458234,
 'training_score': 0.406959873623961}

Algoritmo utilizado:
{'estimator_class': 'sklearn.neural_network._multilayer_perceptron.MLPRegressor',
 'estimator_name': 'MLPRegressor'}

Artifacts:
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt

# 5. Seleção de Hiper-Parâmetros

A avaliação e seleção de hiper-parâmetros é um passo bastante importante na escolha do modelo final pois é nessa etapa em que são definidas características que serão utilizadas no treinamento, como: número de vizinhos no kNN, número de camadas numa rede neural, profundidade numa árvore de decisão, entre outras. No fim das contas, são essas escolhas que vão determinar se o modelo vai performar excepcionalmente bem ou muito longe do ideal.

Existem vários algoritmos de otimização de hiper-parâmetros, e diferentes algoritmos se adequam a diferentes tarefas. Como nosso dataset não é tão grande, vamos utilizar o *Random Search*, que é bastante conhecido e não é tão exigente computacionalmente. Faremos a otimização sobre os algoritmos de árvore de decisão, random forest e rede neural vistos anteriormente. 

In [33]:
# Importando o Random Search com cross-validation
from sklearn.model_selection import RandomizedSearchCV

## Decision Tree

In [34]:
# Definindo o Random Search CV. Vamos fornecer o argumento n_iter, que fala quantas configurações de hparams testar:
random_search_cv1 = RandomizedSearchCV( estimator = tree.DecisionTreeRegressor(random_state=10),
                                        param_distributions = { 'max_depth':range(1,10),            # Testando comprimentos máximos de 1 a 10
                                                                'min_samples_leaf':range(1,8),     # O mínimo de amostras por folha, de 1 a 8
                                                                'min_samples_split':range(2,4)} ,   
                                        #scoring='roc_auc', 
                                        cv = 3,
                                        n_iter = 20)

# Realizando a otimização por GridSearch para os dados de cancer de mama:
random_search_cv1.fit(x_train,y_train)

#Vamos ver informações relevantes:
print('Melhor comprimento máximo: {}'.format(random_search_cv1.best_params_['max_depth']))
print('Melhor minimo de amostras por folha: {}'.format(random_search_cv1.best_params_['min_samples_leaf']))
print('Melhor min_samples_split encontrado: {}'.format(random_search_cv1.best_params_['min_samples_split']))
#print('Desempenho AUC-ROC do melhor modelo: {}'.format(round(random_search_cv1.best_score_,3)))
print('Tempo para realizar a otimização:')
%timeit -n 1 -r 1 random_search_cv1.fit(x_train,y_train)

2021/08/17 05:05:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ed399320e96f42b88b789e45353540be', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/08/17 05:05:25 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1,...`
2021/08/17 05:05:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as flo

Melhor comprimento máximo: 4
Melhor minimo de amostras por folha: 5
Melhor min_samples_split encontrado: 2
Tempo para realizar a otimização:


2021/08/17 05:05:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2021/08/17 05:05:27 INFO mlflow.sklearn.utils: Logging the 5 best runs, 15 runs will be omitted.


1 loop, best of 1: 1.1 s per loop


## Random Forest

In [35]:
# Definindo o Random Search CV. Vamos fornecer o argumento n_iter, que fala quantas configurações de hparams testar:
random_search_cv2 = RandomizedSearchCV( estimator = RandomForestRegressor(random_state=10),
                                        param_distributions = { 'max_depth':range(1,10),            # Testando comprimentos máximos de 1 a 10
                                                                'min_samples_leaf':range(1,8),     # O mínimo de amostras por folha, de 1 a 8
                                                                'min_samples_split':range(2,4)} ,   
                                        #scoring='roc_auc', 
                                        cv = 3,
                                        n_iter = 20)

# Realizando a otimização por GridSearch para os dados de cancer de mama:
random_search_cv2.fit(x_train,y_train)

#Vamos ver informações relevantes:
print('Melhor comprimento máximo: {}'.format(random_search_cv2.best_params_['max_depth']))
print('Melhor minimo de amostras por folha: {}'.format(random_search_cv2.best_params_['min_samples_leaf']))
print('Melhor min_samples_split encontrado: {}'.format(random_search_cv2.best_params_['min_samples_split']))
#print('Desempenho AUC-ROC do melhor modelo: {}'.format(round(random_search_cv2.best_score_,3)))
print('Tempo para realizar a otimização:')
%timeit -n 1 -r 1 random_search_cv2.fit(x_train,y_train)

2021/08/17 05:05:27 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3a6f24fa7e3a4aa7b803324cf9ee010e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/08/17 05:05:27 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impu...`
2021/08/17 05:06:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as flo

Melhor comprimento máximo: 6
Melhor minimo de amostras por folha: 2
Melhor min_samples_split encontrado: 2
Tempo para realizar a otimização:


2021/08/17 05:07:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2021/08/17 05:07:03 INFO mlflow.sklearn.utils: Logging the 5 best runs, 15 runs will be omitted.


1 loop, best of 1: 47.2 s per loop


## Rede Neural

In [ ]:
# Definindo o Random Search CV. Vamos fornecer o argumento n_iter, que fala quantas configurações de hparams testar:
random_search_cv3 = RandomizedSearchCV( estimator = MLPRegressor(random_state=10),
                                        param_distributions = { 'hidden_layer_sizes':[(50,50,50),(50,100,50),(100,1)],
                                                                'alpha': [0.0001, 0.05],
                                                                'learning_rate': ['constant','adaptive'] }, 
                                        #scoring='roc_auc', 
                                        cv = 3,
                                        n_iter = 20)

# Realizando a otimização por GridSearch para os dados de cancer de mama:
random_search_cv3.fit(x_train,y_train)

#Vamos ver informações relevantes:
print('Melhor hidden_layer_sizes: {}'.format(random_search_cv3.best_params_['hidden_layer_sizes']))
print('Melhor alpha: {}'.format(random_search_cv3.best_params_['alpha']))
print('Melhor learning_rate: {}'.format(random_search_cv3.best_params_['learning_rate']))
#print('Desempenho AUC-ROC do melhor modelo: {}'.format(round(random_search_cv3.best_score_,3)))
print('Tempo para realizar a otimização:')
%timeit -n 1 -r 1 random_search_cv3.fit(x_train,y_train)

2021/08/17 04:55:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f306e50457ed4229bb15455c4b3c2bb3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/08/17 04:55:23 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, m...`
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning

Melhor hidden_layer_sizes: (50, 50, 50)
Melhor alpha: 0.0001
Melhor learning_rate: constant
Tempo para realizar a otimização:


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_mu

# 6. Diagnóstico e Aprimoramento

A partir os resultados da seleção, podemos aplicar novamente os algoritmos vistos no tópico 3, agora com os melhores hiper-parâmetros encontrados.